In [42]:
import obspy
from obspy.signal.filter import bandpass
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

#  FUNGSI PEMBACA DATA
def read_seismic_data(mseed_file, xml_file):
    try:
        st = obspy.read(mseed_file)
        inv = obspy.read_inventory(xml_file)  # StationXML
        return st, inv
    except Exception as e:
        messagebox.showerror("Error", f"Error membaca file: {str(e)}")
        return None, None

# Ambil informasi STASIUN dari StationXML
def extract_station_info(xml_file):
    try:
        inv = obspy.read_inventory(xml_file)
        net = inv.networks[0]
        sta = net.stations[0]

        latitude = sta.latitude
        longitude = sta.longitude
        elevation = sta.elevation
        station_code = sta.code

        return latitude, longitude, elevation, station_code

    except Exception as e:
        messagebox.showerror("Error", f"Gagal membaca info stasiun dari XML: {str(e)}")
        return None, None, None, None

#   FILTER & SIMULASI
def filter_data(stream, freqmin, freqmax):
    st_f = stream.copy()
    for tr in st_f:
        tr.data = bandpass(tr.data, freqmin=freqmin, freqmax=freqmax,
                           df=tr.stats.sampling_rate, corners=4, zerophase=True)
    return st_f

def apply_wood_anderson(stream, inv):
    wa_stream = stream.copy()
    
    paz_wa = {
        "poles": [-5.49779 - 5.60886j, -5.49779 + 5.60886j],
        "zeros": [0j, 0j],
        "gain": 1,
        "sensitivity": 1
    }

    for tr in wa_stream:
        tr.simulate(paz_remove=None, paz_simulate=paz_wa, water_level=60)

    return wa_stream

def calculate_amplitude_max(stream):
    result = {}
    for tr in stream:
        result[tr.id] = np.max(np.abs(tr.data))
    return result

#   PERHITUNGAN MAGNITUDO
def calculate_magnitude_local(max_amp, dist_km):
    return np.log10(max_amp) + 3 * np.log10(8 * dist_km) - 2.92

def calculate_magnitude_surface(max_amp, dist_km):
    return np.log10(max_amp) + 1.66 * np.log10(dist_km) + 3.3

def calculate_magnitude_body(max_amp, dist_km):
    return np.log10(max_amp) + 0.75 * np.log10(dist_km) + 2.5

#   PLOT SEISMOGRAM (SCROLLABLE)

# ====== PLOT SEISMOGRAM DI MAIN WINDOW ======
def plot_seismogram():

    mseed_file = entry_mseed.get()
    if not mseed_file:
        messagebox.showwarning("Error", "Harap pilih file MSEED terlebih dahulu.")
        return

    try:
        st = obspy.read(mseed_file)
    except Exception as e:
        messagebox.showerror("Error", f"Gagal membaca MSEED: {str(e)}")
        return

    # --- Jika plot sudah ada, hapus dulu ---
    global plot_canvas
    try:
        plot_canvas.get_tk_widget().destroy()
    except:
        pass

    # --- Frame untuk plot (di main window) ---
    plot_frame = tk.Frame(root, bg="#d6e6f2")
    plot_frame.grid(row=9, column=0, columnspan=3, pady=20)

    # --- Figure untuk seismogram ---
    fig = Figure(figsize=(9, 3.8), dpi=100)   # <<< ukuran lebih kecil & pas
    axes = []

    # Pisahkan komponen
    comps = {"BHZ": None, "BHN": None, "BHE": None}
    for tr in st:
        ch = tr.stats.channel
        if ch.endswith("Z"):
            comps["BHZ"] = tr
        elif ch.endswith("N"):
            comps["BHN"] = tr
        elif ch.endswith("E"):
            comps["BHE"] = tr

    axes_order = ["BHZ", "BHN", "BHE"]

    for i, comp in enumerate(axes_order, 1):
        tr = comps[comp]
        if tr is not None:
            ax = fig.add_subplot(3, 1, i)
            t = np.linspace(0, tr.stats.npts / tr.stats.sampling_rate, tr.stats.npts)

            ax.plot(t, tr.data, linewidth=0.9)
            ax.set_title(f"Seismogram {comp}", fontsize=9)
            ax.set_ylabel("Amplitudo", fontsize=8)

            if i == 3:
                ax.set_xlabel("Waktu (detik)", fontsize=9)

            ax.tick_params(axis='both', labelsize=7)

    fig.tight_layout(pad=2.2)

    # Tampilkan di main window
    plot_canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    plot_canvas.draw()
    plot_canvas.get_tk_widget().pack()

    # =====================================================
    #  SCROLL DENGAN MOUSE (TANPA SCROLLBAR)
    # =====================================================

    def _on_mousewheel(event):
        canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

    # Windows
    canvas.bind_all("<MouseWheel>", _on_mousewheel)
    # Linux
    canvas.bind_all("<Button-4>", lambda e: canvas.yview_scroll(-1, "units"))
    canvas.bind_all("<Button-5>", lambda e: canvas.yview_scroll(1, "units"))

    # Update scroll region
    def update_scroll_region(event):
        canvas.configure(scrollregion=canvas.bbox("all"))

    inner_frame.bind("<Configure>", update_scroll_region)


def calculate_magnitudes():
    mseed_file = entry_mseed.get()
    xml_file = entry_xml.get()

    if not mseed_file or not xml_file:
        messagebox.showwarning("Input Error", "Harap pilih file mseed dan xml.")
        return

    try:
        epicentral_distance_km = float(entry_distance.get())
    except:
        messagebox.showwarning("Input Error", "Jarak epicenter harus angka.")
        return

    st, inv = read_seismic_data(mseed_file, xml_file)
    if st is None:
        return

    # Ambil info stasiun
    lat, lon, elevation, code = extract_station_info(xml_file)

    for item in table_info.get_children():
        table_info.delete(item)

    table_info.insert("", "end", values=(
        f"{lat:.4f}" if lat else "-",
        f"{lon:.4f}" if lon else "-",
        f"{elevation:.2f}" if elevation else "-",
        code if code else "-"
    ))

    st_filtered = filter_data(st, 0.01, 20)
    st_wa = apply_wood_anderson(st_filtered, inv)

    amp_max = calculate_amplitude_max(st_wa)

    text_result.delete(1.0, tk.END)

    for tr_id, amp in amp_max.items():
        if var_magnitude.get() == "local":
            mg = calculate_magnitude_local(amp, epicentral_distance_km)
            text_result.insert(tk.END, f"ML untuk {tr_id}: {mg:.2f}\n")

        elif var_magnitude.get() == "surface":
            mg = calculate_magnitude_surface(amp, epicentral_distance_km)
            text_result.insert(tk.END, f"MS untuk {tr_id}: {mg:.2f}\n")

        elif var_magnitude.get() == "body":
            mg = calculate_magnitude_body(amp, epicentral_distance_km)
            text_result.insert(tk.END, f"MB untuk {tr_id}: {mg:.2f}\n")
# =========================
# FUNGSI BROWSE FILE
# =========================

def browse_mseed():
    filepath = filedialog.askopenfilename(
        title="Pilih file .mseed",
        filetypes=[("MSEED files", "*.mseed"), ("All files", "*.*")]
    )
    if filepath:
        entry_mseed.delete(0, tk.END)
        entry_mseed.insert(0, filepath)


def browse_xml():
    filepath = filedialog.askopenfilename(
        title="Pilih file StationXML (.xml)",
        filetypes=[("XML files", "*.xml"), ("All files", "*.*")]
    )
    if filepath:
        entry_xml.delete(0, tk.END)
        entry_xml.insert(0, filepath)


#   GUI UTAMA
root = tk.Tk()
root.title("Magnitudo + Seismogram Viewer (StationXML)")
root.configure(bg="#d6e6f2")

font_label = ("Times New Roman", 12)
font_button = ("Times New Roman", 10, "bold")

# Input mseed
tk.Label(root, text="Input file (.mseed):", font=font_label, bg="#d6e6f2").grid(row=0, column=0, sticky="e")
entry_mseed = tk.Entry(root, width=50, font=font_label)
entry_mseed.grid(row=0, column=1, padx=10, pady=5)
tk.Button(root, text="Browse", command=browse_mseed, bg="#6d597a", fg="white",
          font=font_button).grid(row=0, column=2)

# Input xml (StationXML)
tk.Label(root, text="Input file StationXML (.xml):", font=font_label, bg="#d6e6f2").grid(row=1, column=0, sticky="e")
entry_xml = tk.Entry(root, width=50, font=font_label)
entry_xml.grid(row=1, column=1, padx=10, pady=5)
tk.Button(root, text="Browse", command=browse_xml, bg="#6d597a", fg="white",
          font=font_button).grid(row=1, column=2)

# Pilih jenis magnitudo
tk.Label(root, text="Pilih jenis magnitudo:", font=font_label, bg="#d6e6f2").grid(row=2, column=0, sticky="e")

frame_radio = tk.Frame(root, bg="#d6e6f2")
frame_radio.grid(row=2, column=1, columnspan=3, sticky="w")

var_magnitude = tk.StringVar(value="local")

tk.Radiobutton(frame_radio, text="ML", variable=var_magnitude, value="local",
               bg="#d6e6f2", font=font_label).pack(side="left", padx=20)

tk.Radiobutton(frame_radio, text="MS", variable=var_magnitude, value="surface",
               bg="#d6e6f2", font=font_label).pack(side="left", padx=20)

tk.Radiobutton(frame_radio, text="MB", variable=var_magnitude, value="body",
               bg="#d6e6f2", font=font_label).pack(side="left", padx=20)


# Jarak
tk.Label(root, text="Jarak epicenter (km):", font=font_label, bg="#d6e6f2").grid(row=3, column=0, sticky="e")
entry_distance = tk.Entry(root, width=20, font=font_label)
entry_distance.grid(row=3, column=1)
entry_distance.insert(0, "50.0")

# Tombol hitung
tk.Button(root, text="Hitung Magnitudo", command=calculate_magnitudes,
          bg="#6d597a", fg="white", font=font_button).grid(row=4, column=1, pady=10)

# Hasil magnitudo
text_result = tk.Text(root, height=6, width=60, font=font_label, bg="#5f6caf")
text_result.grid(row=5, column=0, columnspan=3, padx=10, pady=10)

# Tabel STATION info
tk.Label(root, text="Informasi STASIUN dari XML:", font=font_label, bg="#d6e6f2").grid(row=1, column=0, sticky="w")

columns = ("Latitude", "Longitude", "Elevation", "Station_code")
table_info = ttk.Treeview(root, columns=columns, show="headings", height=3)

table_info.heading("Latitude", text="Latitude")
table_info.heading("Longitude", text="Longitude")
table_info.heading("Elevation", text="Elevasi (m)")
table_info.heading("Station_code", text="Kode Stasiun")

table_info.grid(row=7, column=0, columnspan=3, padx=10, pady=10)

# Tombol Plot
tk.Button(root, text="Plot Seismogram BHZ/BHN/BHE", command=plot_seismogram,
          bg="#5f6caf", fg="white", font=font_button).grid(row=8, column=1, pady=10)

root.mainloop()

In [40]:
import obspy
from obspy.signal.filter import bandpass
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.figure import Figure

# ---------------------------
# Global placeholder for canvas (so we can destroy previous plots)
# ---------------------------
plot_canvas = None

# ---------------------------
# Utility / processing functions
# ---------------------------
def read_seismic_data(mseed_file, xml_file):
    try:
        st = obspy.read(mseed_file)
        inv = obspy.read_inventory(xml_file) if xml_file else None
        return st, inv
    except Exception as e:
        messagebox.showerror("Error", f"Error membaca file: {str(e)}")
        return None, None

def extract_station_info(xml_file):
    try:
        inv = obspy.read_inventory(xml_file)
        net = inv.networks[0]
        sta = net.stations[0]
        return sta.latitude, sta.longitude, sta.elevation, sta.code
    except Exception as e:
        messagebox.showwarning("Warning", f"Gagal membaca info stasiun: {str(e)}")
        return None, None, None, None

def filter_data(stream, freqmin, freqmax):
    st_f = stream.copy()
    for tr in st_f:
        tr.data = bandpass(tr.data, freqmin=freqmin, freqmax=freqmax,
                           df=tr.stats.sampling_rate, corners=4, zerophase=True)
    return st_f

def apply_wood_anderson(stream, inv=None):
    wa_stream = stream.copy()
    paz_wa = {
        "poles": [-5.49779 - 5.60886j, -5.49779 + 5.60886j],
        "zeros": [0j, 0j],
        "gain": 1,
        "sensitivity": 1
    }
    for tr in wa_stream:
        try:
            tr.simulate(paz_remove=None, paz_simulate=paz_wa, water_level=60)
        except Exception:
            # if simulation fails, just keep original trace (don't crash)
            pass
    return wa_stream

def calculate_amplitude_max(stream):
    result = {}
    for tr in stream:
        result[tr.id] = np.max(np.abs(tr.data))
    return result

def calculate_magnitude_local(max_amp, dist_km):
    return np.log10(max_amp) + 3 * np.log10(8 * dist_km) - 2.92

def calculate_magnitude_surface(max_amp, dist_km):
    return np.log10(max_amp) + 1.66 * np.log10(dist_km) + 3.3

def calculate_magnitude_body(max_amp, dist_km):
    return np.log10(max_amp) + 0.75 * np.log10(dist_km) + 2.5

# ---------------------------
# Plot routine (main window, responsive & scrollable)
# ---------------------------
def plot_seismogram():
    global plot_canvas, toolbar_widget

    mseed_file = entry_mseed.get().strip()
    if not mseed_file:
        messagebox.showwarning("Error", "Harap pilih file MSEED terlebih dahulu.")
        return

    try:
        st = obspy.read(mseed_file)
    except Exception as e:
        messagebox.showerror("Error", f"Gagal membaca MSEED: {str(e)}")
        return

    # clear previous plot widgets if any
    for w in plot_container.winfo_children():
        w.destroy()
    plot_container.update_idletasks()

    # make figure with modest height so subplots don't overlap
    # width will visually adapt inside the scrollable area
    fig = Figure(figsize=(10, 4), dpi=100)
    fig.patch.set_facecolor("#ffffff")

    # collect components
    comps = {"BHZ": None, "BHN": None, "BHE": None}
    for tr in st:
        ch = tr.stats.channel.upper()
        if ch.endswith("Z"):
            comps["BHZ"] = tr
        elif ch.endswith("N"):
            comps["BHN"] = tr
        elif ch.endswith("E"):
            comps["BHE"] = tr

    axes_list = []
    axes_order = ["BHZ", "BHN", "BHE"]
    n_plots = sum(1 for c in axes_order if comps[c] is not None)
    if n_plots == 0:
        messagebox.showwarning("Warning", "Tidak ditemukan komponen BHZ/BHN/BHE pada file.")
        return

    # adjust figure height based on number of components (reasonable scaling)
    height_per_trace = 1.0  # inches per subplot
    fig.set_size_inches(10, max(2.8, n_plots * height_per_trace + 1.2))

    for i, comp in enumerate(axes_order, start=1):
        tr = comps[comp]
        if tr is None: 
            continue
        ax = fig.add_subplot(n_plots, 1, len(axes_list) + 1)
        t = np.linspace(0, tr.stats.npts / tr.stats.sampling_rate, tr.stats.npts)
        ax.plot(t, tr.data, linewidth=0.7)
        ax.set_title(f"Seismogram {comp}", fontsize=10)
        ax.set_ylabel("Amplitude", fontsize=9)
        ax.tick_params(axis='both', labelsize=8)
        if len(axes_list) < n_plots - 1:
            # hide x tick labels for all but last plot to reduce clutter
            ax.set_xticklabels([])
        else:
            ax.set_xlabel("Time (s)", fontsize=9)
        axes_list.append(ax)

    # tighten layout with extra padding so titles/labels don't overlap
    fig.tight_layout(pad=2.2)

    # embed into scrollable canvas area
    plot_canvas = FigureCanvasTkAgg(fig, master=plot_container)
    plot_widget = plot_canvas.get_tk_widget()
    plot_widget.pack(fill="both", expand=True)

    # navigation toolbar (optional)
    toolbar = NavigationToolbar2Tk(plot_canvas, plot_container)
    toolbar.update()
    toolbar.pack(fill="x", side="bottom")
    toolbar_widget = toolbar

    plot_canvas.draw()

# ---------------------------
# Magnitude calculation & update UI
# ---------------------------
def calculate_magnitudes():
    mseed_file = entry_mseed.get().strip()
    xml_file = entry_xml.get().strip()

    if not mseed_file or not xml_file:
        messagebox.showwarning("Input Error", "Harap pilih file mseed dan xml.")
        return

    try:
        epicentral_distance_km = float(entry_distance.get())
    except Exception:
        messagebox.showwarning("Input Error", "Jarak epicenter harus angka.")
        return

    st, inv = read_seismic_data(mseed_file, xml_file)
    if st is None:
        return

    lat, lon, elev, code = extract_station_info(xml_file)
    for item in table_info.get_children():
        table_info.delete(item)
    table_info.insert("", "end", values=(
        f"{lat:.4f}" if lat else "-",
        f"{lon:.4f}" if lon else "-",
        f"{elev:.2f}" if elev else "-",
        code if code else "-"
    ))

    st_filtered = filter_data(st, 0.01, 20)
    st_wa = apply_wood_anderson(st_filtered, inv)
    amp_max = calculate_amplitude_max(st_wa)

    text_result.config(state="normal")
    text_result.delete(1.0, tk.END)
    for tr_id, amp in amp_max.items():
        if var_magnitude.get() == "local":
            mg = calculate_magnitude_local(amp, epicentral_distance_km)
            text_result.insert(tk.END, f"ML untuk {tr_id}: {mg:.2f}\n")
        elif var_magnitude.get() == "surface":
            mg = calculate_magnitude_surface(amp, epicentral_distance_km)
            text_result.insert(tk.END, f"MS untuk {tr_id}: {mg:.2f}\n")
        elif var_magnitude.get() == "body":
            mg = calculate_magnitude_body(amp, epicentral_distance_km)
            text_result.insert(tk.END, f"MB untuk {tr_id}: {mg:.2f}\n")
    text_result.config(state="disabled")

# ---------------------------
# Browse helpers (keamanan: insert path only if chosen)
# ---------------------------
def browse_mseed():
    fp = filedialog.askopenfilename(title="Pilih file .mseed", filetypes=[("MSEED", "*.mseed"), ("All files", "*.*")])
    if fp:
        entry_mseed.delete(0, tk.END)
        entry_mseed.insert(0, fp)

def browse_xml():
    fp = filedialog.askopenfilename(title="Pilih StationXML (.xml)", filetypes=[("XML", "*.xml"), ("All files", "*.*")])
    if fp:
        entry_xml.delete(0, tk.END)
        entry_xml.insert(0, fp)

# ---------------------------
# Main GUI
# ---------------------------
root = tk.Tk()
root.title("Magnitudo + Seismogram Viewer (StationXML)")
root.configure(bg="#f5f7f9")     # modern light grey-white
root.geometry("1100x700")

font_label = ("Segoe UI", 10)
font_button = ("Segoe UI", 9, "bold")

# Use grid everywhere
padx = 12
pady = 6

# Row 0: file inputs
tk.Label(root, text="Input file (.mseed):", font=font_label, bg="#f5f7f9").grid(row=0, column=0, sticky="e", padx=padx, pady=pady)
entry_mseed = tk.Entry(root, width=70, font=font_label)
entry_mseed.grid(row=0, column=1, columnspan=2, sticky="we", padx=padx, pady=pady)
ttk.Button(root, text="Browse", command=browse_mseed).grid(row=0, column=3, padx=padx, pady=pady)

tk.Label(root, text="Input file StationXML (.xml):", font=font_label, bg="#f5f7f9").grid(row=1, column=0, sticky="e", padx=padx, pady=pady)
entry_xml = tk.Entry(root, width=70, font=font_label)
entry_xml.grid(row=1, column=1, columnspan=2, sticky="we", padx=padx, pady=pady)
ttk.Button(root, text="Browse", command=browse_xml).grid(row=1, column=3, padx=padx, pady=pady)

# Row 2: magnitude options and distance
tk.Label(root, text="Pilih jenis magnitudo:", font=font_label, bg="#f5f7f9").grid(row=2, column=0, sticky="e", padx=padx)
var_magnitude = tk.StringVar(value="local")
frame_radio = tk.Frame(root, bg="#f5f7f9")
frame_radio.grid(row=2, column=1, sticky="w", padx=padx)
tk.Radiobutton(frame_radio, text="ML", variable=var_magnitude, value="local", bg="#f5f7f9").pack(side="left", padx=8)
tk.Radiobutton(frame_radio, text="MS", variable=var_magnitude, value="surface", bg="#f5f7f9").pack(side="left", padx=8)
tk.Radiobutton(frame_radio, text="MB", variable=var_magnitude, value="body", bg="#f5f7f9").pack(side="left", padx=8)

tk.Label(root, text="Jarak epicenter (km):", font=font_label, bg="#f5f7f9").grid(row=2, column=1, sticky="e", padx=padx)
entry_distance = tk.Entry(root, width=12)
entry_distance.grid(row=2, column=2, sticky="w", padx=padx)
entry_distance.insert(0, "0.0")

# Row 3: magnitude button & result box
ttk.Button(root, text="Hitung Magnitudo", command=calculate_magnitudes).grid(row=3, column=1, pady=pady)
text_result = tk.Text(root, height=6, width=70, bg="#eef2f7", font=font_label)
text_result.grid(row=4, column=0, columnspan=4, padx=padx, pady=(6,12))
text_result.config(state="disabled")

# Row 5: station info table
tk.Label(root, text="Informasi STASIUN dari XML:", font=font_label, bg="#f5f7f9").grid(row=5, column=0, sticky="w", padx=padx, pady=(0,6))
columns = ("Latitude", "Longitude", "Elevation", "Station_code")
table_info = ttk.Treeview(root, columns=columns, show="headings", height=1)
for col in columns:
    table_info.heading(col, text=col)
table_info.grid(row=6, column=0, columnspan=4, sticky="we", padx=padx, pady=(0,12))

# Row 7: Plot button
ttk.Button(root, text="Plot Seismogram BHZ/BHN/BHE", command=plot_seismogram).grid(row=7, column=1, pady=(0,12))

# Row 8: plot container (scrollable area)
# We'll use a Frame that can expand; the content (matplotlib canvas) will be packed inside it.
plot_outer = tk.Frame(root, bg="#ffffff", bd=1, relief="flat")
plot_outer.grid(row=8, column=0, columnspan=4, sticky="nsew", padx=padx, pady=(6,12))

# Make the main grid expandable so plot area grows with window
root.grid_rowconfigure(8, weight=1)
root.grid_columnconfigure(1, weight=1)
root.grid_columnconfigure(2, weight=1)

# Inner container where matplotlib will be inserted
plot_container = tk.Frame(plot_outer, bg="#ffffff")
plot_container.pack(fill="both", expand=True)

# Start GUI
root.mainloop()
